In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import util

In [3]:
dir_data = Path('/Users/josh/Documents/Harvard/GLM/sabatinilab-glm/data/old-data-version/raw-new/Figure_1_2')
dir_output = Path('/Users/josh/Desktop/example_output_folder')

lst_dict_inputdata = [
    {'session_id': 'WT63_11082021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT69_12192021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT69_12192021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},
]

dir_output.mkdir(parents=True, exist_ok=True)

columnName_alignment_trial_start = 'photometryCenterInIndex'
columnName_alignment_trial_end = 'photometrySideOutIndex'

# Note: Alignment values of 0 for Matlab-indexed trial tables will be treated as "no-data" values
# and and -1 for Python-indexed trial tables. Matlab-indexed trial tables should only have values
# >= 0 in and >= -1 in Python.
trial_index_columns = [
    'photometryCenterInIndex',
    'photometryCenterOutIndex',
    'photometrySideInIndex',
    'photometrySideOutIndex',
]

trial_num_columns = [
    'leftRewardProb',
    'rightRewardProb',
]

trial_cat_columns = [
    'nTrial',
    # 'hasData',
    'wasRewarded',
    'word',
]

signal_dense_columns = [
    'Ch1',
    'Ch2',
    'Ch5',
    'Ch6',
]

signal_sparse_columns = [
    # 'centerOcc',
    # 'centerIn',
    # 'centerOut',
    # 'rightOcc',
    'rightIn',
    # 'rightOut',
    # 'rightLick',
    # 'leftOcc',
    'leftIn',
    # 'leftOut',
    # 'leftLick',
    # 'SEnt'
]

In [7]:
for dict_inputdata in lst_dict_inputdata:
    df_signal = pd.read_csv(dict_inputdata['filepath_signal'])
    df_trial = pd.read_csv(dict_inputdata['filepath_trial'])
    df_trial['nTrial'] = np.arange(df_trial.shape[0])

    nrows = df_signal.shape[0]
    
    df_trial = df_trial[(df_trial[trial_index_columns] == 0).sum(axis=1) == 0]
    df_trial_indices = df_trial[trial_index_columns]
    
    matrix_signal_sparse = util.Matrix.from_pd_dense_num(df_signal[signal_sparse_columns])
    matrix_signal_dense = util.Matrix.from_pd_dense_num(df_signal[signal_dense_columns])
    matrix_trial_inx = util.Matrix.from_pd_sparse_num(nrows, indices=df_trial_indices - 1, values=None, sparse_value=0)
    matrix_trial_num = util.Matrix.from_pd_sparse_num(nrows, indices=df_trial_indices - 1, values=df_trial[trial_num_columns], sparse_value=np.nan)
    matrix_trial_cat = util.Matrix.from_pd_sparse_cat(nrows, indices=df_trial_indices - 1, values=df_trial[trial_cat_columns], sparse_value=np.nan)
    
    matrix_all_preds = util.Matrix([matrix_signal_sparse, matrix_signal_dense, matrix_trial_num, matrix_trial_cat])

    # # print(matrix_all_preds.vectors['centerOcc'])

    
    # mtc = matrix_trial_cat.to_pd()['nTrial']
    # display(mtc[(~mtc.isna()).sum(axis=1) > 0])
    # # display(matrix_trial_cat.to_pd()['nTrial'])
    # display(matrix_trial_inx.to_pd())

In [12]:
matrix_all_preds.to_pd()

,rightIn,leftIn,Ch1,Ch2,Ch5,Ch6,"(leftRewardProb, photometryCenterInIndex)","(rightRewardProb, photometryCenterInIndex)","(leftRewardProb, photometryCenterOutIndex)","(rightRewardProb, photometryCenterOutIndex)",...,"((wasRewarded, 0), photometrySideOutIndex)","((wasRewarded, 1), photometrySideOutIndex)","((word, AA), photometrySideOutIndex)","((word, AB), photometrySideOutIndex)","((word, Aa), photometrySideOutIndex)","((word, Ab), photometrySideOutIndex)","((word, aA), photometrySideOutIndex)","((word, aB), photometrySideOutIndex)","((word, aa), photometrySideOutIndex)","((word, ab), photometrySideOutIndex)"
0,0,0,-2.515587,0,0.087224,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,-1.660977,0,-0.159638,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,-2.155949,0,-0.408390,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,-0.868106,0,-0.799495,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,-0.477200,0,-0.422909,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29845,0,0,-2.426426,0,0.590765,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29846,0,0,-2.861054,0,-1.745552,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29847,0,0,-2.314100,0,-0.742046,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29848,0,0,-1.455169,0,-0.879324,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
break

In [ ]:
matrix_all_preds.vectors

In [ ]:
[type(m) for m in matrix_all_preds.vectors.values()]

In [ ]:
mappd = matrix_all_preds.to_pd()

In [ ]:
list(mappd.columns)

In [ ]:
# mappd[('word', 'photometrySideOutIndex')].dropna()
matrix_all_preds.vectors['rightIn']

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(mappd[[]], mappd['Ch5'])

In [ ]:
display(matrix_signal_dense.to_pd())
display(matrix_trial_inx.to_pd())
display(matrix_trial_num.to_pd())
display(matrix_trial_cat.to_pd())